In [ ]:
import json

In [ ]:
hybrid_questions_train= json.load(open('/home/turning/temp/Hybrid_QA_MM/HybridQA/released_data/train.json','r'))
hybrid_questions_dev= json.load(open('/home/turning/temp/Hybrid_QA_MM/HybridQA/released_data/dev.json','r'))
hybrid_questions_test= json.load(open('/home/turning/temp/Hybrid_QA_MM/HybridQA/released_data/test.json','r'))

In [ ]:
hybrid_questions_combined = []
hybrid_questions_combined.extend(hybrid_questions_train)
hybrid_questions_combined.extend(hybrid_questions_dev)
hybrid_questions_combined.extend(hybrid_questions_test)
# check for duplicates in question_id


In [ ]:
hybrid_questions_combined

In [ ]:
all_tables = {}
cells_to_links_each_table = {}
different_tables = []
for question in hybrid_questions_combined:
    table_id = question['table_id']
    table = json.load(open('/home/turning/temp/Hybrid_QA_MM/WikiTables-WithLinks/tables_tok/'+table_id+'.json','r'))
    # formatting the table so that it can be  put through pandas
    table_formatted = []
    cells_to_link = []
    
    
    header_list = table['header']
   
    data = table['data']
    #insert the headers in index 0 of data
    data.insert(0,header_list)
    # print(data)
    
    for row_id, row in enumerate(data): 
        row_formatted = []
        row_formatted_links = []
        for cell_id, cell in enumerate( row) :
            # print( cell[1])
            row_formatted_links.append(cell[1])
            row_formatted.append(cell[0])
        table_formatted.append(row_formatted)
        cells_to_link.append(row_formatted_links)
    if table_id =="Cambridge-Isanti_Bluejacket_Football_0":
        print( table_formatted , cells_to_link)
    all_tables[table_id] = {'table':table_formatted,'url': table['url'],'cells_to_link':cells_to_link}
    # cells_to_links_each_table[table_id] = cells_to_link
    

In [ ]:
len(all_tables
)

In [ ]:
import jsonlines
all_tables_jsonl = []
for key in all_tables.keys():
    new_dict ={}
    new_dict['table_id'] = key
    new_dict['table'] = all_tables[key]['table']
    new_dict['url'] = all_tables[key]['url']
    new_dict['cells_to_link'] = all_tables[key]['cells_to_link']
    if key =="Cambridge-Isanti_Bluejacket_Football_0":
        print( str(new_dict))
    all_tables_jsonl.append(new_dict)
    

with jsonlines.open('/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/all_tables.jsonl', mode='w') as writer:
    writer.write_all(all_tables_jsonl)

In [2]:
#code for link revision ids 
import urllib.parse
import urllib.request

base_url = "https://en.wikipedia.org/w/api.php"
title = "List_of_ambassadors_of_the_United_States_to_Trinidad_and_Tobago"  # Replace with the desired title

params = {
    'action': 'query',
    'titles': title,
    'prop': 'revisions',
    'rvlimit': '500',
    'rvstart': '2020-05-05T07:59:00Z',
    'rvdir': 'older',
    'format': 'json',
    'redirects': '1'
}

# Construct the URL with query parameters
url = f"{base_url}?{urllib.parse.urlencode(params)}"
print(url)
# Send an HTTP GET request to the URL
response = urllib.request.urlopen(url)

# Read and print the response
data = response.read()
print(data.decode('utf-8'))

URL_template = 'https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z&rvdir=older&format=json&redirects=1'

https://en.wikipedia.org/w/api.php?action=query&titles=List_of_ambassadors_of_the_United_States_to_Trinidad_and_Tobago&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1
{"batchcomplete":"","query":{"normalized":[{"from":"List_of_ambassadors_of_the_United_States_to_Trinidad_and_Tobago","to":"List of ambassadors of the United States to Trinidad and Tobago"}],"pages":{"17444611":{"pageid":17444611,"ns":0,"title":"List of ambassadors of the United States to Trinidad and Tobago","revisions":[{"revid":945958389,"parentid":944343074,"minor":"","user":"Lepricavark","timestamp":"2020-03-17T04:53:12Z","comment":"/* top */added short description"},{"revid":944343074,"parentid":941184799,"user":"InternetArchiveBot","timestamp":"2020-03-07T07:01:04Z","comment":"Rescuing 1 sources and tagging 0 as dead.) #IABot (v2.0"},{"revid":941184799,"parentid":910818105,"user":"Postcard Cathy","timestamp":"2020-02-17T02:43:49Z","comment":"/* List of ambassadors */"}

# Now since we got this in particular format we now get the revision ids 

In [ ]:
# to get the revision ids we need to put api requests to wikiapi 
revision_ids_queries = {}
for table_id in all_tables:
    url = URL_template.format(title = all_tables[table_id]['url'].split("en.wikipedia.org/wiki/")[1])
    revision_ids_queries[table_id] = url

In [ ]:
json.dump(revision_ids_queries,open('/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_inputs/revision_ids_queries.json','w'))

#### Now we need to query these and get revision ids  using the asyncio code `Hybrid_QA_MM/src/asycnio_codes/asyncio_revision_id.py`

In [ ]:
# the revision ids are stored in the file /home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_outputs/revision_ids.json
revision_ids= json.load(open('/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_outputs/revision_ids.json','r'))
# Now we need to get the htmls for each revision id so that we can scrape the tables from them 
revision_ids_2_link_of_wikipedia_page = {}
for query in revision_ids:
    revision_id = revision_ids[query]
    revision_ids_2_link_of_wikipedia_page[query] = f"https://en.wikipedia.org/w/index.php?title=Page_Title&oldid={revision_id}"

In [ ]:
json.dump(revision_ids_2_link_of_wikipedia_page,open('/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_inputs/revision_ids_2_link_of_wikipedia_page.json','w'))

Now we run the asyncio script


### Now we scrape teh htmls from 
`/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_outputs/revision_ids_2_link_of_wikipedia_page.json`

In [ ]:
import json
results_html_path = "/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_outputs/revision_ids_2_link_of_wikipedia_page.json"
results_html = json.load(open(results_html_path,'r'))

In [ ]:
# now we scrape tables fro the htmls
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
list_no_tables = []
linkstables = {}
for i in tqdm(results_html):
    link = i
    html = results_html[i]
    soup = BeautifulSoup(html, 'lxml')
    tables = soup.find_all('table')
    linkstables[link] = tables
    if len( tables)==0:
        list_no_tables.append(link)
        print(link)
        print(results_html[i])
        # print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA") 

In [ ]:
len(linkstables)

In [ ]:
list(linkstables.keys())[:3]

### Now we have scraped the tables we need to make a dict {url : tables }

In [ ]:
t1= json.load(open("/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/asyncio_inputs/revision_ids_2_link_of_wikipedia_page.json", "r"))


In [ ]:
urls_to_tables = {}
x= 0
for i in t1:
    if t1[i] in linkstables:
        urls_to_tables[i] = linkstables[t1[i]]
    else:
        print(i)
        x+=1
        print("Links not found")

In [ ]:
print ("""
Total number of links not found: {}
       length of urls_to_tables: {}
       length of linkstables: {}.
""".format(x,len(urls_to_tables),len(linkstables)))
       

In [ ]:
#find all nested tables 
url_to_new_tables = {}
for url, tables in urls_to_tables.items():
    new_tables = []
    for table in tables:
        n_tables = table.find_all('table')
        if len(n_tables)>0:
            new_tables.extend(n_tables)
        else:
            new_tables.append(table)
    url_to_new_tables[url] = new_tables

In [ ]:
# we will store the table in htmls in the directory /home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/tables
id = 0
path = "/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/tables"
url_to_table_id = {}
from tqdm import tqdm
import os 
if not os.path.exists("tables"):
    os.makedirs("tables")
for url , tables in tqdm(url_to_new_tables.items()):
    for table in tables: 
        table_id = id
        url_to_table_id.setdefault(url, [])
        url_to_table_id[url].append(table_id)
        id+=1
        table_html = str(table)
        
        with open(f"{path}/{table_id}.html", "w") as f:
            f.write(table_html)
        


In [ ]:
json.dump(url_to_table_id, open("/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/url_to_table_id_merged.json", "w"))

In [ ]:
# get url to table id mapping
import json
url_to_table_id = json.load(open("/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/url_to_table_id_merged.json", "r"))
URL_template = 'https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z&rvdir=older&format=json&redirects=1'

In [ ]:
url_to_table_id

In [ ]:
# now we need to get the table ids for each url and the table array from 
import pandas as pd
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from itertools import product
from tqdm import tqdm
file_path = "/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/all_tables.jsonl"
rev_cnt=0
most_similar_tables = {}
iii=0
# Open the JSONL file and read line by line
redirect_titles = []
with open(file_path, "r") as jsonl_file:
    for line  in tqdm(jsonl_file) :
       
        json_data = json.loads(line)
        
        # Now you can work with the JSON data as a Python dictionary
        # print(json_data)  # You can replace this with your processing logic
        page_url = json_data['url']
        # print(page_url)
        
        page_url = URL_template.format(title = page_url.split("en.wikipedia.org/wiki/")[1])
        id = json_data['table_id']
        # raw_title = page_url.split("en.wikipedia.org/wiki/")[1]
        table_1_df  = pd.DataFrame(json_data['table'])
        words = set()
        for i in range(table_1_df.shape[0]):
            for j in range(table_1_df.shape[1]):
                cell_value = str(table_1_df.iloc[i,j]).lower()
                if len(cell_value.split())>1:
                    n=2 
                    ngramss = set(ngrams(word_tokenize(cell_value), n))
                    words.update(set(ngramss))
                else :
                    words.add(cell_value)
                
        
        table_id_with_max_similarity = None
        max_similarity = 0
        
        if page_url in url_to_table_id:
            for table_ids in url_to_table_id[page_url]:
                try : 
                    # print(f"/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/tables/{table_ids}.html")
                    table_2_df = pd.read_html(f"/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/tables/{table_ids}.html")[0]
                    # print("hi")
                    words2 = set()
                    for i in range(table_2_df.shape[0]):
                        for j in range(table_2_df.shape[1]):
                            cell_value = str(table_2_df.iloc[i,j]).lower()
                            if len(cell_value.split())>1:
                                n=2
                                ngras = set(ngrams(word_tokenize(cell_value), n))
                                words2.update(set(ngras))
                            else :
                                words2.add(cell_value)
                    similarity = len(words.intersection(words2))/len(words.union(words2))
                    if similarity>max_similarity:
                        max_similarity = similarity
                        table_id_with_max_similarity = table_ids
                except Exception as e:
                    print(e)
                    print( "Path : ", f"/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/tables/{table_ids}.html")
                    iii+=1 
                    continue
                # print( "Path : ", f"tables/{table_ids}.html")
                # ngrams_table_1 = set()
                # ngrams_table_2 = set()
                # n=2 
                # ngrams_1 = set()
                # for i in range(table_1_df.shape[0]):
                #     for j in range(table_1_df.shape[1]):
                #         cell_value = table_1_df.iloc[i,j]
                #         if type(cell_value)==str:
                #             if len(cell_value.split())>1:
                #                 ngrams_1.update(set(ngrams(cell_value.split(), n)))
        else : 
            print(page_url)
        most_similar_tables[id] =(page_url,table_id_with_max_similarity, max_similarity) 
        # print( table_id_with_max_similarity, max_similarity, page_url)
        # print(table_1_df)

                
        
        

In [ ]:
len(most_similar_tables)

In [ ]:
json.dump(most_similar_tables,open("/home/turning/temp/Hybrid_QA_MM/outputs_table_scraping/most_similar_tables.json","w"))

In [1]:
import json
# import pandas as pd
from tqdm import tqdm
# from bs4 import BeautifulSoup
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/most_similar_tables.json") as f:
    most_similar_tables = json.load(f)

In [2]:
print(list(most_similar_tables.items())[:10])

[('List_of_mosques_in_Afghanistan_0', ['https://en.wikipedia.org/w/api.php?action=query&titles=List_of_mosques_in_Afghanistan&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z&rvdir=older&format=json&redirects=1', 47673, 0.8188976377952756]), ('1999_Kyalami_Superbike_World_Championship_round_0', ['https://en.wikipedia.org/w/api.php?action=query&titles=1999_Kyalami_Superbike_World_Championship_round&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z&rvdir=older&format=json&redirects=1', 3820, 0.8275862068965517]), ('List_of_ECAC_Hockey_Player_of_the_Year_0', ['https://en.wikipedia.org/w/api.php?action=query&titles=List_of_ECAC_Hockey_Player_of_the_Year&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z&rvdir=older&format=json&redirects=1', 53078, 0.45]), ('List_of_Zulu_War_Victoria_Cross_recipients_0', ['https://en.wikipedia.org/w/api.php?action=query&titles=List_of_Zulu_War_Victoria_Cross_recipients&prop=revisions&rvlimit=500&rvstart=2020-09-09T07%3A59%3A00Z

In [3]:
# open the  html of the table and get table_d:link_list for each cell 
from bs4 import BeautifulSoup
from collections import defaultdict
table_links = {}
for feta_id, table_info in tqdm(most_similar_tables.items()):
    page_url = table_info[0]
    table_id = table_info[1]
    if table_id is None:
        continue
    table_links[table_id] = defaultdict(list)
    with open(f"/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/tables/{table_id}.html", "r") as f:
        html = f.read()
        soup = BeautifulSoup(html, 'lxml')
        tables = soup.find_all('table')
        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                for cell in cells:
                    links = cell.find_all('a')
                    for link in links:
                        if link.get('href') is not None and  "/wiki/" in link.get('href') and "redlink=1" not in link.get('href') and "Special:Upload?wpDestFile" not in link.get('href'):
                            table_links[table_id][link.get('href')].append(link.text)

100%|██████████| 12378/12378 [05:42<00:00, 36.15it/s]


In [5]:
for table in table_links:
    for link in table_links[table]:
        table_links[table][link] = list(set(table_links[table][link]))
        # if len(table_links[table][link])>1:
        #     print(table_links[table][link], link)
        for i , text in enumerate(table_links[table][link]):
            table_links[table][link][i] = text.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ")
        table_links[table][link] = sorted(list(set(table_links[table][link])) , key = lambda x: len(x), reverse=True)
            

In [6]:
list(table_links.items())[:10]

[(47673,
  defaultdict(list,
              {'/wiki/Abdul_Rahman_Mosque': ['Abdul Rahman Mosque'],
               '/wiki/Kabul_Province': ['Kabul Province'],
               '/wiki/Kabul': ['Kabul'],
               '/wiki/Shrine_of_the_Cloak': ['Friday Mosque of Kandahar'],
               '/wiki/File:Mosque_in_Kandahar-2011.jpg': [''],
               '/wiki/Kandahar_Province': ['Kandahar Province'],
               '/wiki/Kandahar': ['Kandahar'],
               '/wiki/Muhammad': ['Muhammad'],
               '/wiki/File:Sin_foto.svg': [''],
               '/wiki/Great_Mosque_of_Herat': ['Great Mosque of Herat'],
               '/wiki/File:Friday_Mosque_in_Herat,_Afghanistan.jpg': [''],
               '/wiki/Herat_Province': ['Herat Province'],
               '/wiki/Herat': ['Herat'],
               '/wiki/Fire_temple': ['Fire temples'],
               '/wiki/Shrine_of_Ali': ['Shrine of Ali'],
               '/wiki/File:Mazar-e_sharif_-_Steve_Evans.jpg': [''],
               '/wiki/Balkh_Pr

In [7]:
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/table_links_nl.json","w") as f:
    json.dump(table_links,f)


In [1]:
import json
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/table_links_nl.json") as f:
    table_links = json.load(f)

In [2]:
table_links

{'47673': {'/wiki/Abdul_Rahman_Mosque': ['Abdul Rahman Mosque'],
  '/wiki/Kabul_Province': ['Kabul Province'],
  '/wiki/Kabul': ['Kabul'],
  '/wiki/Shrine_of_the_Cloak': ['Friday Mosque of Kandahar'],
  '/wiki/File:Mosque_in_Kandahar-2011.jpg': [''],
  '/wiki/Kandahar_Province': ['Kandahar Province'],
  '/wiki/Kandahar': ['Kandahar'],
  '/wiki/Muhammad': ['Muhammad'],
  '/wiki/File:Sin_foto.svg': [''],
  '/wiki/Great_Mosque_of_Herat': ['Great Mosque of Herat'],
  '/wiki/File:Friday_Mosque_in_Herat,_Afghanistan.jpg': [''],
  '/wiki/Herat_Province': ['Herat Province'],
  '/wiki/Herat': ['Herat'],
  '/wiki/Fire_temple': ['Fire temples'],
  '/wiki/Shrine_of_Ali': ['Shrine of Ali'],
  '/wiki/File:Mazar-e_sharif_-_Steve_Evans.jpg': [''],
  '/wiki/Balkh_Province': ['Balkh Province'],
  '/wiki/Mazar-i-Sharif': ['Mazar-i-Sharif'],
  '/wiki/Sakhi_Shrine': ['Sakhi Shrine'],
  '/wiki/File:Jalalabad_minarets.jpg': [''],
  '/wiki/Nangarhar_Province': ['Nangarhar Province'],
  '/wiki/Jalalabad': ['Ja

In [3]:
with open("/home/suyash/final_repo/HybridQA_Tab_MM/outputs_combined_old&new/link_to_single_image.json", "r") as f:
    link_to_single_image = json.load(f)

with open("/home/suyash/final_repo/HybridQA_Tab_MM/outputs_combined_old&new/download_paths.json") as f:
    download_paths = json.load(f)

link_to_image_path = {}
for link in link_to_single_image:
    if link_to_single_image[link] in download_paths:
        link_to_image_path[link] = download_paths[link_to_single_image[link]]
    else:
        # print(link_to_single_image[link])
        pass

In [4]:
list(link_to_image_path.items())[:10]

[('https://en.wikipedia.org/wiki/Coventry',
  '/home/suyash/final_repo/HybridQA_Tab_MM/retried_remaining_outputs/hybrid_qa/Broadgate_and_Precincts_-_geograph.org.uk_-_554599.jpg'),
 ('https://en.wikipedia.org/wiki/Andrea_Checchi',
  '/scratch/jainit/hybrid_qa_table_images/Biraghin_(film_1946)_Andrea_Checchi_(2)_(cropped).png'),
 ('https://en.wikipedia.org/wiki/Ralph_Towner',
  '/scratch/jainit/hybrid_qa_table_images/Towner387.jpg'),
 ('https://en.wikipedia.org/wiki/Greenwich_Park',
  '/home/suyash/final_repo/HybridQA_Tab_MM/retried_remaining_outputs/hybrid_qa/GreenwichParcRoyal.jpg'),
 ('https://en.wikipedia.org/wiki/La_Plagne',
  '/scratch/jainit/hybrid_qa_table_images/LAPLAGNE_Vertical_Couleurs.jpg'),
 ('https://en.wikipedia.org/wiki/Summit_League',
  '/scratch/jainit/hybrid_qa_table_images/247px-Summit_League_logo.svg.png'),
 ('https://en.wikipedia.org/wiki/The_Summit_League',
  '/scratch/jainit/hybrid_qa_table_images/247px-Summit_League_logo.svg.png'),
 ('https://en.wikipedia.org/w

In [5]:
# now we need to get images for each table 
# by running the pipeline 
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/table_links_nl.json") as f:
    table_links = json.load(f)


In [6]:
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/all_links_dict.json") as f:
    og_links_to_edited_links = json.load(f)

In [7]:
import re
def replace_with_condition(cell_string, replace_str, replacement_str):
    # Define the regular expression pattern to match replace_str with non-alphanumeric or no adjacent characters
    cell_string = " " + cell_string + " "
    pattern = r'([^a-zA-Z0-9])' + re.escape(replace_str) + r'([^a-zA-Z0-9])'
    
    # Use re.sub to replace all matches in the cell_string with replacement_str with case-insensitive matching
    result = re.sub(pattern, r'\1' + replacement_str + r'\2', cell_string, flags=re.IGNORECASE)
    is_unchanged = cell_string==result
    return (result.strip(), is_unchanged)

In [ ]:
import re 
def calculate_alpha_ratio(input_string):
    
    # remove all spaces
    input_string = input_string.replace(" ", "")
    alpha_count = sum(c.isalpha() for c in input_string)
    non_alpha_count = len(input_string) - alpha_count
    
    if non_alpha_count == 0:
        return 1
    ratio = alpha_count / non_alpha_count
    # print("Ratio", ratio)
    return ratio

In [34]:
import json
import re
from urllib.parse import unquote
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
from copy import deepcopy

table_links = json.load(open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/table_links_nl.json", "r"))
most_similar_tables = json.load(open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/most_similar_tables.json", "r"))
file_path = "/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/all_tables.jsonl"

with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/outputs_table_scraping/table_id_to_idx.json") as f:
    table_id_to_idx = json.load(f)

f  = open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/final_dataset_unfiltered/tables.jsonl", "w")

i=0
j=0 

links_not_in_html = []

image_id_to_wiki_link = {}
image_id_to_image_path = {}
image_id_to_og_string = {}

strings_to_links = {}
all_links = set()
image_id_string_replaced= {}

hits_link = 0
total_links = 0
links_with_no_image = 0
links_with_no_image_list = []

temp_new_tables_arrays = []

NOTES_ARRAY = ["note", "comment", "description"]

with open(file_path, "r") as jsonl_file:
    for line  in tqdm(jsonl_file) :
        json_data = json.loads(line)
        
        page_url = json_data['url']
        table = deepcopy(json_data['table'])
        cell_links = json_data['cells_to_link']
        ttable_id  = json_data['table_id'] # Since table_ids are strings, we are converting them to indexes and storing these values
        
        # now we get the table_id for this fetad_id adn get teh table and then get the links for each cell
    
        table_id = str(most_similar_tables[str(json_data['table_id'])][1])
        
        ttable_id_idx = table_id_to_idx[ttable_id]
        new_tables = deepcopy(table)
        table_headers = table[0]

        for row_id, row in enumerate(table):
            for cell_id , cell in enumerate(row):
                cell = cell.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                if cell is not None and cell != "":
                    
                    image_id_for_image_string = 0
                    links = cell_links[row_id][cell_id]
                    
                    for link in links:
                        
                        if link.startswith("/wiki/"):
                            link = "https://en.wikipedia.org" + link

                        if link not in link_to_image_path:
                            links_with_no_image += 1
                            links_with_no_image_list.append(link)
                            continue
                        
                        is_found = False
                        
                        if table_id in table_links:
                            curr_table_links = table_links[table_id]
                            scraped_table_texts = curr_table_links.get(link, [])

                            for scraped_table_text in scraped_table_texts:
                                scraped_table_text = scraped_table_text.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                                alpha_ratio =  calculate_alpha_ratio(scraped_table_text)
                                
                                if alpha_ratio<0.5 and len(scraped_table_text)<4:
                                    continue
                                
                                if scraped_table_text != "" and len(scraped_table_text)>3 and link in link_to_image_path and "/wiki/" in link and any([elem in table_headers[cell_id].lower() for elem in NOTES_ARRAY]) == 0: #and  bool(re.search(r'[a-zA-Z]', text)):
                                
                                    img_string = f"{{IMG-{{HTQ-{ttable_id_idx}-{row_id}-{cell_id}-{image_id_for_image_string}}}}}"
                                    cell, is_unchanged = replace_with_condition(cell, scraped_table_text, img_string)

                                    if not is_unchanged:
                                        image_id_to_image_path[img_string] = link_to_image_path[link]
                                        image_id_to_og_string[img_string] = scraped_table_text
                                        image_id_to_wiki_link[img_string] = link
                                        image_id_for_image_string+=1
                                        is_found = True
                                        break
                                    
                        if not is_found:
                            
                            link_text = link.split('/')[-1].replace("_", " ").lower().strip().replace("\n", " ").replace("\t", " ").replace("\r", " ")
                            if link_text != "":
                                link_text = link_text.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                                img_string = f"{{IMG-{{HTQ-{ttable_id_idx}-{row_id}-{cell_id}-{image_id_for_image_string}}}}}"
                                if link_text != "" and len(link_text)>3 and link in link_to_image_path and "/wiki/" in link and any([elem in table_headers[cell_id].lower() for elem in NOTES_ARRAY]) == 0: 
                                    cell, is_unchanged = replace_with_condition(cell, link_text, img_string)
                                    new_tables[row_id][cell_id] = new_tables[row_id][cell_id].replace(link_text, img_string)
                                    
                                    if not is_unchanged:
                                        image_id_to_image_path[img_string] = link_to_image_path[link]
                                        image_id_to_og_string[img_string] = link_text
                                        image_id_to_wiki_link[img_string] = link
                                        image_id_for_image_string+=1
                                        is_found = True
                        
                        if is_found:
                            hits_link+=1
                        total_links+=1
                
                new_tables[row_id][cell_id] = cell
                        
        new_table_json_data = {
            "table_id": ttable_id,
            "table_idx": ttable_id_idx,
            "table_array": new_tables,
            "original_table_array": table,
            "url": json_data['url'],
        }
        f.write(json.dumps(new_table_json_data)+"\n")
        temp_new_tables_arrays.append(new_tables)

f.close()
print("Total links: ", total_links)
print("Hits: ", hits_link)
print("Links with no image: ", links_with_no_image)

# Original results without adding the extra leftout images
# Total links:  279183
# Hits:  250533
# Links with no image:  267864


# Modified results
# Total links:  431882
# Hits:  252222
# Links with no image:  115165

# Before adding filter on alpha ratio
# 12378it [02:06, 97.65it/s] 
# Total links:  431882
# Hits:  252222
# Links with no image:  115165

12378it [02:01, 101.59it/s]

Total links:  431882
Hits:  238703
Links with no image:  115165


In [35]:
def generate_table_string(table_array):
    table_string = ""

    for row_idx, row in enumerate(table_array):
        for col_idx, cell in enumerate(row):
            cell = cell.replace("\t", " ").replace("\n", " ").replace("\\n", " ").replace("\\t", " ").replace("|", " ")
            table_string = table_string  + cell + " | "
        table_string = table_string + "\n"

    return table_string

def get_all_image_tags(text):
    pattern = r"\{IMG-\{.*?\}\}"
    matches = re.findall(pattern, text)
    return matches


for table in temp_new_tables_arrays:
    table_string = generate_table_string(table)
    image_tags = get_all_image_tags(table_string)
    for img_tag in image_tags:
        num_occ = img_tag.count("IMG-{HTQ")
        if num_occ > 1:
            print(img_tag)
            # print(image_id_to_og_string['{IMG-{HTQ-3489-16-2-1}'])
            print("AAAAAAAAAAAAA")

unique_strs = set()
for img_id, og_string in image_id_to_og_string.items():
    has_number = any(ch.isdigit() for ch in og_string)
    if has_number:
        unique_strs.add(og_string)
    # if img_id.startswith("{IMG-{HTQ-3489-16"):
unique_strs = list(unique_strs)
# sort on basis of length
unique_strs.sort(key = lambda x: len(x), )

In [36]:
print(unique_strs)

['7rph', 'fab1', 'f-86', 'fm-7', 'e915', '1q84', '8:30', 'e909', 'bbc2', 'e223', 'abc1', 'e414', 'tv18', 'e406', 'e713', 'b4-4', 'a-91', '2pac', 't-90', 'gp10', 'svt2', 'vt-6', 'oh 7', '3sat', 'rtp2', 'jg 7', 'b-67', 'lh 4', '3.9g', 'gpl3', 'e416', 'abc2', 'vp-1', 'vt-9', 'gp16', '3abn', 'be-4', '7two', 'fox8', 'e210', '5-ht', 'i-40', 'oh 5', 'mtv2', 'vp-5', 'be-3', 'tsn2', 'e704', 'itv1', 'hv71', 'rté2', 'e417', 'i-75', 't-62', 'vh-1', 't-72', 'c-5a', 'b-29', 'vp-8', 'ub40', 't-64', 'e906', 'gp11', 'kbs2', 'e710', '16.5', 'cz12', 'e703', 'm-86', 'dna²', 't-80', 'pt-6', 'e211', 'acl2', '2ne1', '9:06', 'rtp1', 'p-8a', 'e702', 'op12', 'p90x', 'm-84', 'abc3', 'vp-4', 'tvp1', 'gpl2', 'vp-9', 'mtv3', 'e904', 'ctv 2', 'hsl 4', 'pp-70', '7mate', 'bt-42', 'win32', 'net 5', 'e1105', 'su-76', 'more4', 'sts-8', 'ogk-2', 'orf 2', 'vp-45', 'hsl 3', 'pc-98', 'ny 74', 'sat.1', '6teen', 'ak-47', 'tg-16', '10 km', '2-8-2', 'spg-9', 'vp-69', 'nc 37', 'e1401', 'vr-54', 'vt-22', 'sts-9', 'vp-46', 'abc 2',

In [39]:
with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/final_dataset_unfiltered/image_id_to_wiki_link.json", "w") as f:
    json.dump(image_id_to_wiki_link, f, indent=4)

with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/final_dataset_unfiltered/image_id_to_image_path.json", "w") as f:
    json.dump(image_id_to_image_path, f, indent=4)

with open("/home/suyash/final_repo/HybridQA_Tab_MM/Hybrid_QA_MM/final_dataset_unfiltered/image_id_to_og_string.json", "w") as f:
    json.dump(image_id_to_og_string, f, indent=4)


In [38]:
# with open("/home/suyash/final_repo/HybridQA_Tab_MM/outputs_remaining_images/links_with_no_image.json", "w") as f:
#     json.dump(links_with_no_image_list, f, indent=4)